# Coursera Capstone Project - The Battle of the Neighbourhoods

Since the covid-19 outbreak, many establishments have been forced out of business. With new vaccines on the way, however, the hospitality sector's future looks more promising than before. Entrepreneurs will race to fill the gaps left behind by the venues that sadly could not survive the outbreak. 

In this notebook, we will identify the type of hospitality sector establishment (i.e. restaurants, cafés,...) that are most likely to be successful in the Amsterdam city centre. In addition, we will find the best neighbourhood for it to be established in.

First we import all the necessary dependencies:

In [1]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import display 
import matplotlib.pyplot as plt

## Data acquisition

#### Below I extract a table from wikipedia with information about the 9 neighbourhoods of the Amsterdam-Centrum borough.

In [2]:
url = 'https://nl.wikipedia.org/wiki/Amsterdam-Centrum'
table = pd.read_html(url, attrs={"class":"wikitable sortable"}, header=0)[0]

In [3]:
table = table[['Buurtnaam']]
table.columns = ['Neighbourhood']

In [4]:
table['Neighbourhood'][9] = 'Oostelijke Eilanden'
table

,Neighbourhood
0,Burgwallen-Oude Zijde
1,Burgwallen-Nieuwe Zijde
2,Grachtengordel-West
3,Grachtengordel-Zuid
4,Nieuwmarkt en Lastage
5,Haarlemmerbuurt
6,Jordaan
7,De Weteringschans
8,Weesperbuurt en Plantage
9,Oostelijke Eilanden


#### Using the geopy package we obtain the coordinates of each neighbourhood

In [5]:
geolocator = Nominatim(user_agent="to_explorer")
lats = []
lngs = []
for name in table['Neighbourhood']:
    address = name
    loc = geolocator.geocode(address)
    lats.append(loc.latitude)
    lngs.append(loc.longitude)

In [6]:
table['Latitude'] = lats
table['Longitude'] = lngs

In [7]:
table

,Neighbourhood,Latitude,Longitude
0,Burgwallen-Oude Zijde,52.371946,4.896103
1,Burgwallen-Nieuwe Zijde,52.373706,4.889922
2,Grachtengordel-West,52.370837,4.885478
3,Grachtengordel-Zuid,52.367650,4.887376
4,Nieuwmarkt en Lastage,52.372177,4.906351
5,Haarlemmerbuurt,52.382441,4.887193
6,Jordaan,52.375416,4.881096
7,De Weteringschans,52.361147,4.886864
8,Weesperbuurt en Plantage,52.365224,4.910656
9,Oostelijke Eilanden,52.371553,4.921840


#### Let's visualise the neighbourhoods' locations:

In [8]:
geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode('Amsterdam')

In [9]:
amsmap = folium.Map(location=[location.latitude, location.longitude], zoom_start = 14)

In [10]:
for lat, lng, neighbourhood in zip(table['Latitude'], table['Longitude'], table['Neighbourhood']):
    label = str(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(amsmap)
    
amsmap

In [11]:
table.drop(2, inplace=True) # drop due to closeness to other neighbourhoods

In [12]:
table

,Neighbourhood,Latitude,Longitude
0,Burgwallen-Oude Zijde,52.371946,4.896103
1,Burgwallen-Nieuwe Zijde,52.373706,4.889922
3,Grachtengordel-Zuid,52.367650,4.887376
4,Nieuwmarkt en Lastage,52.372177,4.906351
5,Haarlemmerbuurt,52.382441,4.887193
6,Jordaan,52.375416,4.881096
7,De Weteringschans,52.361147,4.886864
8,Weesperbuurt en Plantage,52.365224,4.910656
9,Oostelijke Eilanden,52.371553,4.921840


In [13]:
amsmap = folium.Map(location=[location.latitude, location.longitude], zoom_start = 14)
for lat, lng, neighbourhood in zip(table['Latitude'], table['Longitude'], table['Neighbourhood']):
    label = str(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(amsmap)
    
amsmap

#### Here we get the top venues in each neighbourhood from Foursquare:

In [14]:
CLIENT_ID = 'VHASLCTGROSODBBLMJLZWAN5CNV0WY5SIRIMOWJI1HJZ4USS' # your Foursquare ID
CLIENT_SECRET = 'L32FTJCPUHGQQW1CJBFFG0I5UWNS310UX350WU4HHVTEC1ZU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
venues = getNearbyVenues(table['Neighbourhood'], table['Latitude'], table["Longitude"], radius = 400)

#### We only want the venues from the hospitality sector, i.e. bars, restaurants, etc.

In [17]:
test = venues[(venues['Venue Category'].str.find('Restaurant') > -1) 
                | (venues['Venue Category'].str.find('Bar') > -1)
                | (venues['Venue Category'].str.find('Pub') > -1) 
                | (venues['Venue Category'].str.find('Joint') > -1)
                | (venues['Venue Category'].str.find('Bistro') > -1) 
                | (venues['Venue Category'].str.find('Place') > -1) 
                | (venues['Venue Category'].str.find('Diner') > -1)]

In [18]:
pd.unique(test['Venue Category'])

array(['Cocktail Bar', 'Restaurant', 'Bar', 'Seafood Restaurant',
       'French Restaurant', 'Chinese Restaurant', 'Wine Bar',
       'Asian Restaurant', 'Salon / Barbershop', 'Snack Place',
       'Hotel Bar', 'Malay Restaurant', 'Swiss Restaurant',
       'Ramen Restaurant', 'Thai Restaurant', 'Sandwich Place',
       'Burger Joint', 'Beer Bar', 'Italian Restaurant', 'Bistro',
       'Gay Bar', 'Pub', 'Lebanese Restaurant', 'American Restaurant',
       'Dutch Restaurant', 'Karaoke Bar', 'Vegetarian / Vegan Restaurant',
       'Pizza Place', 'Tibetan Restaurant', 'Mexican Restaurant',
       'Indonesian Restaurant', 'Mediterranean Restaurant',
       'African Restaurant', 'Comfort Food Restaurant', 'Diner',
       'Tapas Restaurant', 'Burrito Place', 'Greek Restaurant',
       'Argentinian Restaurant', 'Salad Place', 'Afghan Restaurant',
       'Spanish Restaurant', 'Juice Bar', 'Japanese Restaurant',
       'Latin American Restaurant', 'Indian Restaurant',
       'Middle Eastern Re

In [19]:
#remove barbershop
remove = test[test['Venue Category'] == 'Salon / Barbershop']

In [20]:
venuemap = folium.Map(location=[location.latitude, location.longitude], zoom_start = 14)
for lat, lng, neighbourhood in zip(test['Venue Latitude'], test['Venue Longitude'], test['Venue']):
    label = str(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venuemap)
    
venuemap

In [21]:
test.drop(remove.index.values, inplace=True)

C:\Users\Hette\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
pd.unique(test['Venue Category'])

array(['Cocktail Bar', 'Restaurant', 'Bar', 'Seafood Restaurant',
       'French Restaurant', 'Chinese Restaurant', 'Wine Bar',
       'Asian Restaurant', 'Snack Place', 'Hotel Bar', 'Malay Restaurant',
       'Swiss Restaurant', 'Ramen Restaurant', 'Thai Restaurant',
       'Sandwich Place', 'Burger Joint', 'Beer Bar', 'Italian Restaurant',
       'Bistro', 'Gay Bar', 'Pub', 'Lebanese Restaurant',
       'American Restaurant', 'Dutch Restaurant', 'Karaoke Bar',
       'Vegetarian / Vegan Restaurant', 'Pizza Place',
       'Tibetan Restaurant', 'Mexican Restaurant',
       'Indonesian Restaurant', 'Mediterranean Restaurant',
       'African Restaurant', 'Comfort Food Restaurant', 'Diner',
       'Tapas Restaurant', 'Burrito Place', 'Greek Restaurant',
       'Argentinian Restaurant', 'Salad Place', 'Afghan Restaurant',
       'Spanish Restaurant', 'Juice Bar', 'Japanese Restaurant',
       'Latin American Restaurant', 'Indian Restaurant',
       'Middle Eastern Restaurant', 'Taco Place

#### Let's see how many venues have been received for each neighbourhood:

In [23]:
bart = pd.DataFrame(test.groupby('Neighborhood').count()['Venue'])
bart.columns = ['# venues']
bart

,# venues
Neighborhood,
Burgwallen-Nieuwe Zijde,19
Burgwallen-Oude Zijde,32
De Weteringschans,24
Grachtengordel-Zuid,39
Haarlemmerbuurt,26
Jordaan,48
Nieuwmarkt en Lastage,12
Oostelijke Eilanden,5
Weesperbuurt en Plantage,21


In [24]:
table['# restaurants'] = bart.values
table.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [25]:
test.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Burgwallen-Oude Zijde,52.371946,4.896103,Rosalia's Menagerie,52.371678,4.899174,Cocktail Bar
1,Burgwallen-Oude Zijde,52.371946,4.896103,The Lobby,52.371159,4.893661,Restaurant
2,Burgwallen-Oude Zijde,52.371946,4.896103,Bar Jones,52.371729,4.893703,Bar
3,Burgwallen-Oude Zijde,52.371946,4.896103,Bridges Restaurant,52.370818,4.895087,Seafood Restaurant
4,Burgwallen-Oude Zijde,52.371946,4.896103,Bougainville,52.372370,4.893111,French Restaurant


#### Now, we one-hot encode the venue category column, so that we can analyse it's frequencies and such.

In [26]:
onehot = pd.get_dummies(test[['Venue Category']], prefix="", prefix_sep="")
onehot.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Bar,Beer Bar,Bistro,Burger Joint,Burrito Place,...,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
henko = test['Neighborhood'] 
try:
    onehot.insert(0, 'Neighborhood', henko)
except ValueError:
    None
onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Bar,Beer Bar,Bistro,Burger Joint,...,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar
0,Burgwallen-Oude Zijde,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Burgwallen-Oude Zijde,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Burgwallen-Oude Zijde,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Burgwallen-Oude Zijde,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Burgwallen-Oude Zijde,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### We would like to see the occurence of each venues category, so that we can establish how popular they are.

In [28]:
grouped = onehot.groupby('Neighborhood').sum().reset_index()
groupedT = grouped.T
cols = groupedT.loc['Neighborhood']
groupedT.drop('Neighborhood', inplace=True)
groupedT.columns = cols
groupedT

Neighborhood,Burgwallen-Nieuwe Zijde,Burgwallen-Oude Zijde,De Weteringschans,Grachtengordel-Zuid,Haarlemmerbuurt,Jordaan,Nieuwmarkt en Lastage,Oostelijke Eilanden,Weesperbuurt en Plantage
Afghan Restaurant,0,0,0,0,0,1,0,0,0
African Restaurant,0,0,0,0,0,0,1,0,0
American Restaurant,0,0,0,1,0,0,0,0,0
Argentinian Restaurant,0,0,0,0,1,0,0,0,0
Asian Restaurant,0,1,1,2,0,0,0,0,0
Bar,4,11,4,4,8,12,4,0,5
Beer Bar,2,1,0,0,1,0,0,0,0
Bistro,0,1,0,0,0,1,0,0,0
Burger Joint,0,1,1,1,0,0,0,0,1
Burrito Place,0,0,0,0,1,0,0,0,0


In [29]:
tonkert = pd.DataFrame(grouped.sum()).reset_index(drop=False)
tonkert.drop(0, inplace=True)
tonkert.reset_index(drop=True, inplace=True)
tonkert.columns = ['Venue Category', 'Count']
display(tonkert)

,Venue Category,Count
0,Afghan Restaurant,1
1,African Restaurant,1
2,American Restaurant,1
3,Argentinian Restaurant,1
4,Asian Restaurant,4
5,Bar,52
6,Beer Bar,4
7,Bistro,2
8,Burger Joint,4
9,Burrito Place,1


We only want the venues that aren't too popular, nor too niche:

In [30]:
mask = (2 < tonkert['Count']) & (tonkert['Count'] < 11)

In [31]:
tonkert[mask]

,Venue Category,Count
4,Asian Restaurant,4
6,Beer Bar,4
8,Burger Joint,4
10,Chinese Restaurant,6
11,Cocktail Bar,7
16,French Restaurant,9
20,Hotel Bar,5
22,Indonesian Restaurant,3
34,Pizza Place,6
35,Pub,5


Finally, we want to see how many of these desirable venues are in each neighbourhood.

In [32]:
ponkie = groupedT.loc[tonkert[mask]['Venue Category'].values]
ponkie.loc[['Asian Restaurant', 'Burger Joint', 'Chinese Restaurant', 'Indian Restaurant', 'Indonesian Restaurant', 'Japanese Restaurant', 'Pizza Place', 'Ramen Restaurant', 'Seafood Restaurant', 'Snack Place', 'Thai Restaurant', 'Vegetarian / Vegan Restaurant']]

C:\Users\Hette\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


Neighborhood,Burgwallen-Nieuwe Zijde,Burgwallen-Oude Zijde,De Weteringschans,Grachtengordel-Zuid,Haarlemmerbuurt,Jordaan,Nieuwmarkt en Lastage,Oostelijke Eilanden,Weesperbuurt en Plantage
Asian Restaurant,0,1,1,2,0,0,0,0,0
Burger Joint,0,1,1,1,0,0,0,0,1
Chinese Restaurant,1,3,0,0,0,0,1,0,1
Indian Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Indonesian Restaurant,0,0,0,1,0,1,0,0,1
Japanese Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pizza Place,0,0,0,1,0,1,1,0,3
Ramen Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seafood Restaurant,0,1,0,1,1,2,0,1,0
Snack Place,0,1,0,0,1,1,0,1,1


## Observations

From this analysis, we can make a couple of observations.

The most popular venues were, by far, bars, French and Italian restaurants and sandwich places. Due to the high competition, it is not recommended to set up these types of establishments. The numerous niche (meaning 1 or 2 establishments in total) restaurants and bars are also not recommended, as their customer base might already be satisfied by the venues that are already in place. 
The recommended types of establishments are those that are neither too prevalent already, nor too niche. Among these types are various types of bars, but these are not recommended because there are already numerous bars in the city centre. In the table above, the number of venues in the recommended categories in each neighbourhood is shown. There are very few venues of these types in Oostelijke Eilanden and Nieuwmarkt en Lastage. However, these neighbourhoods have low venue count in general, which indicates that there might not be a big enough market for restaurants in these neighbourhoods.

Of the remaining neighbourhoods, especially Burgwallen-Nieuwe Zijde and Haarlemmerbuurt seem promising. Burgwallen-Nieuwe Zijde does not have a high overall venue count, but in terms of the recommended types of establishments it is quite empty. Any of the recommended types, except perhaps those that are already present in the neighbourhood, are therefore likely to succeed. 
Haarlemmerbuurt is in a similar position, but in this neighbourhood there are no Asian restaurants at all. This is advantageous, because, for example, a Japanese and a Chinese restaurant in the same neighbourhood might have some overlap in their customer base and will therefore have to compete for the customers in that overlap. Any Asian restaurant in the Haarlemmerbuurt neighbourhood would be the first and would therefore be able to claim the whole potential customer base for itself. 

There is another option, which is less promising than the previous two but nonetheless has some potential. The Jordaan neighbourhood has a high venue count, which means there is a big potential customer base and thus the potential for high rewards, and it also has a gap: there is no vegetarian / vegan restaurant. There are other types of restaurants not present in the Jordaan, but these are all similar to other venue categories that are already present in the neighbourhood. The vegetarian / vegan restaurant has no such competing categories. This is an interesting option that could be explored further.

